In [ ]:
#@title Copyright 2024 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
%%writefile requirements.txt
gdacs-api
google-cloud-pubsub
google-auth

Writing requirements.txt


In [ ]:
%%capture
!pip install -r requirements.txt

In [ ]:
%%writefile main.py

from gdacs.api import GDACSAPIReader
from google.cloud import pubsub_v1
import base64
import google.auth
import json
import logging

# Set up auth.
credentials, project_id = google.auth.default()

# Instantiates a Pub/Sub client
publisher = pubsub_v1.PublisherClient()

# Publishes a message to a Cloud Pub/Sub topic.
def publish_events(request):
    request_json = request.get_json(silent=True) or {}

    client = GDACSAPIReader()
    events = client.latest_events().features

    # If true, don't publish, just log.
    debug = request_json.get("debug")
    topic_name = 'brazil-fire-events' # @param {"type":"string"}

    # References an existing topic
    topic_path = publisher.topic_path(project_id, topic_name)

    # Two styles of filtering. Add more!
    brazil_events = filter(lambda event: any(country['countryname'] == 'Brazil' for country in event['properties']['affectedcountries']), events)
    brazil_fire_events = [event for event in brazil_events if 'fire' in event['properties']['name']]

    message_count = 0
    for event in brazil_fire_events:
      event_id = event['properties']['eventid']

      message_json = json.dumps(event)
      message_bytes = message_json.encode("utf-8")

      # Log the value and return.
      if debug:
        debug_message = f"GDACS event ID {event_id}: {message_json}"
        print(debug_message)
        logging.info(debug)
        message_count += 1
        continue

      # Publishes a message
      try:
        publish_future = publisher.publish(topic_path, data=message_bytes)
        publish_future.result()  # Verify the publish succeeded
        message_count += 1
        logging.info(f"Message published for event {event_id}")
      except Exception as e:
        logging.error(f"Error publishing event {e}")

    print(f"Published {message_count} messages.")
    return

In [ ]:
import json
import importlib
import main
importlib.reload(main)
from flask import Request

from google.colab import auth
PROJECT_ID = "mdewitt-earthengine" # @param {"type":"string"}
auth.authenticate_user(project_id=PROJECT_ID)

data = {
    "debug": True
}

request = Request.from_values(
    headers={'Content-Type': 'application/json'},
    data=json.dumps(data),
    method="POST")

print(main.publish_events(request))

Published 0 messages.


In [ ]:
!gcloud functions deploy publish_events --runtime python312\
 --allow-unauthenticated\
 --trigger-http\
 --project $PROJECT_ID